In [133]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [134]:
!pip install -q tensorflow-hub

In [135]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [136]:
#load dataset
data = pd.read_csv('processed.cleveland.data', header=None)
data.columns
print(data.head())

     0    1    2      3      4    5    6      7    8    9    10   11   12  13
0  63.0  1.0  1.0  145.0  233.0  1.0  2.0  150.0  0.0  2.3  3.0  0.0  6.0   0
1  67.0  1.0  4.0  160.0  286.0  0.0  2.0  108.0  1.0  1.5  2.0  3.0  3.0   2
2  67.0  1.0  4.0  120.0  229.0  0.0  2.0  129.0  1.0  2.6  2.0  2.0  7.0   1
3  37.0  1.0  3.0  130.0  250.0  0.0  0.0  187.0  0.0  3.5  3.0  0.0  3.0   0
4  41.0  0.0  2.0  130.0  204.0  0.0  2.0  172.0  0.0  1.4  1.0  0.0  3.0   0


In [137]:
#define target and variable 
X = data.drop(13, axis=1)
y = data[13]

In [138]:
# Separate features and target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [139]:
X.replace('?', np.nan, inplace=True)

In [140]:
continuous_cols= X.iloc[:, [0, 3, 4, 7, 9, 11]]
categorical_cols = X.iloc[:, [1, 2, 5, 6, 8, 10, 12]]
print(continuous_cols)
print(categorical_cols)

       0      3      4      7    9    11
0    63.0  145.0  233.0  150.0  2.3  0.0
1    67.0  160.0  286.0  108.0  1.5  3.0
2    67.0  120.0  229.0  129.0  2.6  2.0
3    37.0  130.0  250.0  187.0  3.5  0.0
4    41.0  130.0  204.0  172.0  1.4  0.0
..    ...    ...    ...    ...  ...  ...
298  45.0  110.0  264.0  132.0  1.2  0.0
299  68.0  144.0  193.0  141.0  3.4  2.0
300  57.0  130.0  131.0  115.0  1.2  1.0
301  57.0  130.0  236.0  174.0  0.0  1.0
302  38.0  138.0  175.0  173.0  0.0  NaN

[303 rows x 6 columns]
      1    2    5    6    8    10   12
0    1.0  1.0  1.0  2.0  0.0  3.0  6.0
1    1.0  4.0  0.0  2.0  1.0  2.0  3.0
2    1.0  4.0  0.0  2.0  1.0  2.0  7.0
3    1.0  3.0  0.0  0.0  0.0  3.0  3.0
4    0.0  2.0  0.0  2.0  0.0  1.0  3.0
..   ...  ...  ...  ...  ...  ...  ...
298  1.0  1.0  0.0  0.0  0.0  2.0  7.0
299  1.0  4.0  1.0  0.0  0.0  2.0  7.0
300  1.0  4.0  0.0  0.0  1.0  2.0  7.0
301  0.0  2.0  0.0  2.0  0.0  2.0  3.0
302  1.0  3.0  0.0  0.0  0.0  1.0  3.0

[303 rows x 7 c

In [141]:
# X.replace('?', np.nan, inplace=True)

In [142]:
# Imputers : mean and mode
imp_cat = SimpleImputer(strategy='most_frequent')
imp_cont = SimpleImputer(strategy='mean')

In [143]:
#impute missing values
X_cont_imputed = pd.DataFrame(imp_cont.fit_transform(continuous_cols), columns=continuous_cols.columns)
X_cat_imputed = pd.DataFrame(imp_cat.fit_transform(categorical_cols), columns=categorical_cols.columns)


In [144]:
print(X_cat_imputed)

      1    2    5    6    8    10   12
0    1.0  1.0  1.0  2.0  0.0  3.0  6.0
1    1.0  4.0  0.0  2.0  1.0  2.0  3.0
2    1.0  4.0  0.0  2.0  1.0  2.0  7.0
3    1.0  3.0  0.0  0.0  0.0  3.0  3.0
4    0.0  2.0  0.0  2.0  0.0  1.0  3.0
..   ...  ...  ...  ...  ...  ...  ...
298  1.0  1.0  0.0  0.0  0.0  2.0  7.0
299  1.0  4.0  1.0  0.0  0.0  2.0  7.0
300  1.0  4.0  0.0  0.0  1.0  2.0  7.0
301  0.0  2.0  0.0  2.0  0.0  2.0  3.0
302  1.0  3.0  0.0  0.0  0.0  1.0  3.0

[303 rows x 7 columns]


In [145]:
#one hot encode categorical variable    
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first')
ohe_cat = encoder.fit_transform(X_cat_imputed).toarray()
print(ohe_cat)


[[1. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]

In [146]:

X_cat_imputed.columns = X_cat_imputed.columns.astype(str)
# One-hot encode categorical variable
encoder = OneHotEncoder(drop='first', sparse_output=False)
ohe_cat = encoder.fit_transform(X_cat_imputed)
ohe_cat = pd.DataFrame(ohe_cat, columns=encoder.get_feature_names_out(X_cat_imputed.columns))
print(ohe_cat)

     1_1.0  2_2.0  2_3.0  2_4.0  5_1.0  6_1.0  6_2.0  8_1.0  10_2.0  10_3.0  12_6.0  12_7.0
0      1.0    0.0    0.0    0.0    1.0    0.0    1.0    0.0     0.0     1.0     1.0     0.0
1      1.0    0.0    0.0    1.0    0.0    0.0    1.0    1.0     1.0     0.0     0.0     0.0
2      1.0    0.0    0.0    1.0    0.0    0.0    1.0    1.0     1.0     0.0     0.0     1.0
3      1.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0     0.0     1.0     0.0     0.0
4      0.0    1.0    0.0    0.0    0.0    0.0    1.0    0.0     0.0     0.0     0.0     0.0
..     ...    ...    ...    ...    ...    ...    ...    ...     ...     ...     ...     ...
298    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     1.0     0.0     0.0     1.0
299    1.0    0.0    0.0    1.0    1.0    0.0    0.0    0.0     1.0     0.0     0.0     1.0
300    1.0    0.0    0.0    1.0    0.0    0.0    0.0    1.0     1.0     0.0     0.0     1.0
301    0.0    1.0    0.0    0.0    0.0    0.0    1.0    0.0     1.0     0.0     

In [147]:
#scale continuous variable
scaler = StandardScaler()
X_cont_scaled = scaler.fit_transform(X_cont_imputed)
print(X_cont_scaled)

[[ 0.94872647  0.75752504 -0.2649003   0.01719733  1.08733806 -0.72309499]
 [ 1.39200191  1.61121989  0.76041519 -1.82190531  0.39718162  2.50385129]
 [ 1.39200191 -0.6652997  -0.34228261 -0.90235399  1.34614673  1.42820253]
 ...
 [ 0.28381332 -0.0961698  -2.23814899 -1.51538821  0.13837295  0.35255377]
 [ 0.28381332 -0.0961698  -0.20686358  1.06811312 -0.89686172  0.35255377]
 [-1.82174501  0.35913411 -1.38694368  1.02432497 -0.89686172  0.        ]]


In [ ]:
#one hot encode target variable
from tensorflow.keras.utils import to_categorical
y_encoded = to_categorical(y,num_classes=5)
print(y_encoded)

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [149]:
#combine into one X
X_combined = np.concatenate([ohe_cat, X_cont_scaled ], axis=1)
print(X_combined)

[[ 1.          0.          0.         ...  0.01719733  1.08733806
  -0.72309499]
 [ 1.          0.          0.         ... -1.82190531  0.39718162
   2.50385129]
 [ 1.          0.          0.         ... -0.90235399  1.34614673
   1.42820253]
 ...
 [ 1.          0.          0.         ... -1.51538821  0.13837295
   0.35255377]
 [ 0.          1.          0.         ...  1.06811312 -0.89686172
   0.35255377]
 [ 1.          0.          1.         ...  1.02432497 -0.89686172
   0.        ]]


In [150]:
#split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.3, random_state=42)

In [151]:
best_neurons = 0
best_val_accuracy = 0
best_model = None

# Loop through different hidden layer sizes
for neurons in range(5, 55, 5):
    print(f" {neurons} neurons in the hidden layer")
    
    # Define the model
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(5, activation='softmax')  
    ])
    
    # Compile  model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=25,
        validation_split=0.2,
        verbose=0 
    )
    
    # Print final validation accuracy
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Final validation accuracy with {neurons} neurons: {val_accuracy:.4f}\n")

    # Find the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_neurons = neurons
        best_model = model

print(f"Best number of neurons: {best_neurons} with validation accuracy: {best_val_accuracy:.4f}\n")

 5 neurons in the hidden layer


c:\Users\eva.berepiki\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Final validation accuracy with 5 neurons: 0.3721

 10 neurons in the hidden layer
Final validation accuracy with 10 neurons: 0.5814

 15 neurons in the hidden layer
Final validation accuracy with 15 neurons: 0.6047

 20 neurons in the hidden layer
Final validation accuracy with 20 neurons: 0.5349

 25 neurons in the hidden layer
Final validation accuracy with 25 neurons: 0.6279

 30 neurons in the hidden layer
Final validation accuracy with 30 neurons: 0.5581

 35 neurons in the hidden layer
Final validation accuracy with 35 neurons: 0.6047

 40 neurons in the hidden layer
Final validation accuracy with 40 neurons: 0.6279

 45 neurons in the hidden layer
Final validation accuracy with 45 neurons: 0.5814

 50 neurons in the hidden layer
Final validation accuracy with 50 neurons: 0.6047

Best number of neurons: 25 with validation accuracy: 0.6279



In [152]:
# Evaluate the best model on the test data
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 1.1837
Test Accuracy: 0.5385
